<a href="https://colab.research.google.com/github/RochanaChaturvedi/it-is-all-in-the-name/blob/main/Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Imports

In [ ]:
import pandas as pd
import re
import os
import numpy as np
import pickle 

from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle
from sklearn.model_selection import train_test_split,StratifiedKFold, KFold

import tensorflow as tf
from tensorflow.keras import layers,initializers,regularizers,constraints
from tensorflow.keras.layers import Layer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Multiply,Add,Permute,Dot,Activation,\
    AlphaDropout,RepeatVector, Dense, Bidirectional,LSTM, BatchNormalization, LSTM,SpatialDropout1D,Dropout,\
    GlobalMaxPool1D,GlobalAveragePooling1D,Concatenate,Input,Conv1D, MaxPool1D, Embedding,TimeDistributed,Lambda, Reshape,TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import TensorBoard

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from sklearn.utils import class_weight
from sklearn.feature_selection import RFE
from itertools import compress
from sklearn.metrics import recall_score,log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization

import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer 
from tensorflow.python.framework import ops
from skopt import BayesSearchCV
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.utils import use_named_args

import csv
import gc

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# Global constants

In [ ]:
eps=1e-12
np.random.seed(42)

base_dir= "/content/drive/My Drive/name_to_religion/"
if not os.path.isdir(base_dir):
    os.mkdir(base_dir)

data_dir= base_dir + "data/"
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

model_dir= base_dir+ "model/"
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

# Load Training Data

In [ ]:
# Run Once to save split (for pair programming)
def getTrain_data():
  data=pd.read_csv(data_dir+"REDS_cleaned.csv")
  x=data.name
  xp=data.pname
  xi=data.index
  xs=data.sex
  Y=data.religion
  (x_train1, x_test,xp_train1,xp_test, Y_train1,Y_test,xi_train1,xi_test) = train_test_split(x, xp,Y,xi, stratify=Y, random_state=42,test_size=0.15)
  (x_train, x_val,xp_train,xp_val, Y_train,Y_val,xi_train,xi_val)= train_test_split(x_train1, xp_train1, Y_train1,xi_train1,random_state=42, stratify=Y_train1, test_size=0.17647)
  df=pd.DataFrame({"index":xi_train,"name":x_train,"parent":xp_train,"religion":Y_train})
  df.to_csv(data_dir+'REDS_train.csv')
  df=pd.DataFrame({"index":xi_val,"name":x_val,"parent":xp_val,"religion":Y_val})
  df.to_csv(data_dir+'REDS_val.csv')
  df=pd.DataFrame({"index":xi_test,"name":x_test,"parent":xp_test,"religion":Y_test,"muslim":y_test})
  df.to_csv(data_dir+'REDS_test.csv')

In [ ]:
# Load Data and Pre-process
def load_reds_split():
    data_train=pd.read_csv(data_dir+"REDS_train.csv",index_col="index")
    data_val=pd.read_csv(data_dir+"REDS_val.csv",index_col="index")
    data_test=pd.read_csv(data_dir+"REDS_test.csv",index_col="index")
    for data in [data_train,data_val,data_test]:
      data.name.replace(" ","}{", regex=True,inplace=True)
      data.parent.replace(np.nan, '', regex=True,inplace=True)

      data.parent.replace(" ","}{", regex=True,inplace=True)
      data.name="{"+data.name.astype(str)+"}"
      data.parent="{"+data.parent.astype(str)+"}"
      if(concat_model):
        data.name='#'+data.name.astype(str)+'#'+data.parent.astype(str)+'#' 

    if non_neural==True:
      data_train["religion_id"]= data_train.religion.factorize()[0]
      category_id_df = data_train[['religion', 'religion_id']].drop_duplicates().sort_values('religion_id')
      category_to_id = dict(category_id_df.values)
      id_to_category = dict(category_id_df[['religion_id', 'religion']].values)
      Y_train=data_train["religion"].map(category_to_id)
      Y_val=data_val["religion"].map(category_to_id)
      Y_test=data_test["religion"].map(category_to_id)
      return data_train.name,data_val.name,data_test.name,data_train.parent,data_val.parent,data_test.parent,Y_train,Y_val,Y_test,category_to_id,id_to_category
    else:
      lbe = LabelBinarizer()
      lbe.fit(data_train.religion)
      return data_train.name,data_val.name,data_test.name,data_train.parent,data_val.parent,data_test.parent,data_train.religion,data_val.religion,data_test.religion,lbe

In [ ]:
#Load Secondary dataset and pre-process
def load_upsbm():
  data_test_sbm = pd.read_csv(data_dir+"upsbm_annotated.csv")

  data_test_sbm.familyhead=data_test_sbm.familyhead.str.upper()
  data_test_sbm.fatherhusbandname=data_test_sbm.fatherhusbandname.str.upper()

  data_test_sbm.familyhead.replace("\."," ", regex=True,inplace=True)
  data_test_sbm.familyhead.replace("([A-Z])\\1\\1+","\\1", regex=True,inplace=True)
  data_test_sbm.familyhead.replace("\s+"," ", regex=True,inplace=True)

  data_test_sbm.fatherhusbandname.replace("\."," ", regex=True,inplace=True)
  data_test_sbm.fatherhusbandname.replace("([A-Z])\\1\\1+","\\1", regex=True,inplace=True)
  data_test_sbm.fatherhusbandname.replace("\s+"," ", regex=True,inplace=True)

  data_test_sbm = data_test_sbm.apply(lambda x: x.str.strip(" ") if x.dtype == "object" else x)
  # data_test_sbm.to_csv("/content/drive/My Drive/name_to_religion/data/upsbm_annotated2.csv")
  
  data_test_sbm.familyhead.replace(" ","}{", regex=True,inplace=True)
  data_test_sbm.fatherhusbandname.replace(" ","}{", regex=True,inplace=True)

  y_sbm = data_test_sbm.muslim.map(lambda y:"Muslim" if y==1 else "Others")
  x_sbm="{"+data_test_sbm.familyhead.astype(str)+"}"
  xp_sbm="{"+data_test_sbm.fatherhusbandname.astype(str)+"}"
  if concat_model==True:
    x_sbm="#"+x_sbm+"#"+xp_sbm+"#"
  return x_sbm,xp_sbm,y_sbm

# x_sbm,xp_sbm,y_sbm=load_upsbm()

# Models

## Non-neural: SVM Linear/Logistic Regression

In [ ]:
concat_model=True
non_neural=True

### fine-tuning

In [ ]:
loss_max=100
def objective(space):
  max_ngram=int(space['max_ngram'])
  iter=int(space['iter'])
  vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1,max_ngram))#12))#max_features=40000
  vectorizer.fit(x_train)
  tfidf_matrix_train = vectorizer.transform(x_train)
  tfidf_matrix_val = vectorizer.transform(x_val)

  if classifier=="LOGIT":
    clf = LogisticRegression(class_weight='balanced', random_state=42, n_jobs=-1, C=space['C'],max_iter=iter)# penalty='l2',max_iter=500,multi_class='multinomial')
  else:
    clf = svm.LinearSVC(class_weight='balanced', C= space['C'],max_iter=iter)
    clf = CalibratedClassifierCV(clf) 
 
  clf.fit(tfidf_matrix_train, Y_train)
    
  # Predict on Cross Validation data
  pred = clf.predict_proba(tfidf_matrix_val)
  global loss_max
  loss = log_loss(Y_val, pred)
  out_file = model_dir+'multiclass_'+classifier+'concat'+str(concat_model)+'trials.csv'
  of_connection = open(out_file, 'w')
  writer = csv.writer(of_connection)

  if(loss<=loss_max):
    filename=model_dir+"vectorizer"+str(max_ngram)+".sav"
    pickle.dump(vectorizer,open(filename, 'wb'))
    filename = model_dir+ 'multiclass_'+classifier+'concat'+str(concat_model)+'.sav'
    pickle.dump(clf, open(filename, 'wb'))
    loss_g=loss
    print(classifier+" "+str(concat_model)+"loss:"+str(loss)+"C:"+str(space['C'])+"ngram:"+str(max_ngram))
  return {'loss': loss , 'status': STATUS_OK}
    

In [ ]:
for classifier in ["LOGIT","SVM"]:
  for concat_model in [True,False]:
    gc.collect()
    x_train,x_val, x_test, xp_train,xp_val, xp_test,Y_train,Y_val, Y_test,category_to_id,id_to_category = load_reds_split() 
    if concat_model==False:
        data1=pd.DataFrame({"name":x_train,"religion":Y_train})
        data2=pd.DataFrame({"name":xp_train,"religion":Y_train})#.apply(lambda x: x.split(" ")[0])})
        data2=data2[data2["name"].notna()]
        data2=data2[data2.name!=""]
        d=pd.concat([data1,data2],axis=0)
        x_train=d.name
        Y_train=d.religion

    space={'C':hp.uniform('C',0,100),'max_ngram':hp.quniform('x_max_ngram',1,12,1),'iter':hp.quniform('iter',60,120,1)}
    trials = Trials()
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=10,
                trials=trials)
    
    print(classifier,concat_model, best)

### Training

In [ ]:
non_neural=True
for classifier in ["SVM"]:
  for concat_model in [False]:
    if classifier=="LOGIT":
      if concat_model:
        opt_C=33.38829440124558
        max_iter=105
        max_ngram=10
      else:
        opt_C= 64.48692955599071
        max_iter= 114
        max_ngram= 12
    else:
      if concat_model:
        opt_C=8.472897550492439
        max_iter=1000
        max_ngram=10
      else:
        opt_C=79.52865215798708
        max_iter=1000
        max_ngram=11
    print(classifier,concat_model)
    gc.collect()
    x_train,x_val, x_test, xp_train,xp_val, xp_test,Y_train,Y_val, Y_test,category_to_id,id_to_category = load_reds_split() 

    if concat_model==False:
        data1=pd.DataFrame({"name":x_train,"religion":Y_train})
        data2=pd.DataFrame({"name":xp_train,"religion":Y_train})
        data2=data2[data2["name"].notna()]
        data2=data2[data2.name!=""]
        d=pd.concat([data1,data2],axis=0)
        x_train=d.name
        Y_train=d.religion

    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1,max_ngram))
    vectorizer.fit(x_train)
        
    tfidf_matrix_train = vectorizer.transform(x_train)
    tfidf_matrix_val = vectorizer.transform(x_val)
    tfidf_matrix_test = vectorizer.transform(x_test)

    if classifier=="LOGIT":
        clf = LogisticRegression(verbose=2,class_weight='balanced', random_state=42, n_jobs=-1, C=opt_C,max_iter=max_iter)
    else:
        clf = svm.LinearSVC(verbose=2,class_weight='balanced', C= opt_C, max_iter=max_iter,random_state=42)
    clf.fit(tfidf_matrix_train, Y_train)

    pickle.dump(clf,open(model_dir+'model_multiclass_'+classifier+'_concat_'+str(concat_model)+'.sav',"wb"))
    pickle.dump(vectorizer,open(model_dir+'vectorizer_multiclass_'+classifier+'_concat_'+str(concat_model)+'.sav',"wb"))

### Testing

In [ ]:
non_neural=True
gc.collect()
x_train,x_val, x_test, xp_train,xp_val, xp_test,Y_train,Y_val, Y_test,category_to_id,id_to_category = load_reds_split() 
x_sbm,xp_sbm,y_sbm=load_upsbm()

for classifier in ["LOGIT","SVM"]:
  for concat_model in [True,False]:
    print(classifier,concat_model)

    vectorizer = pickle.load(open(model_dir+'vectorizer_multiclass_'+classifier+'_concat_'+str(concat_model)+'.sav','rb'))
    clf = pickle.load(open(model_dir+'model_multiclass_'+classifier+'_concat_'+str(concat_model)+'.sav','rb'))
  
    tfidf_matrix_val = vectorizer.transform(x_val)
    tfidf_matrix_test = vectorizer.transform(x_test)
    tfidf_matrix_sbm= vectorizer.transform(x_sbm)

#reds val
    y_pred_val = clf.predict(tfidf_matrix_val)
    df=pd.DataFrame({"familyhead":x_val,"parent":xp_val,"religion":Y_val,
                 "predicted":pd.Series(y_pred_val).map(id_to_category)})
    # df.to_csv(data_dir+'val_predictions_multiclass_'+classifier+'_concat_'+str(concat_model)+'.csv')

    tagset=id_to_category.values()
    print(classifier)
    print(concat_model)
    print('REDS IN SAMPLE Accuracy of classifier level 1: {:.2f}'.format(accuracy_score(Y_val,y_pred_val)))
    print(classification_report(Y_val, y_pred_val,digits=4,target_names=tagset))
    conf_mat = confusion_matrix(Y_val,y_pred_val)
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(conf_mat, annot=True, fmt='d',
                xticklabels=category_to_id.keys(), yticklabels=category_to_id.keys())
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

#reds_test
    y_pred_test = clf.predict(tfidf_matrix_test)
    df = pd.DataFrame({"familyhead":x_test,"parent":xp_test,"religion":Y_test,
                 "predicted":pd.Series(y_pred_test).map(id_to_category)})
    # df.to_csv(data_dir+'test_predictions_multiclass_'+classifier+'_concat_'+str(concat_model)+'.csv')

    print('REDS out of SAMPLE Accuracy of classifier level 1: {:.2f}'.format(accuracy_score(Y_test,y_pred_test)))
    print(classification_report(Y_test, y_pred_test,digits=4,target_names=tagset))
    conf_mat = confusion_matrix(Y_test,y_pred_test)
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(conf_mat, annot=True, fmt='d',
                xticklabels=category_to_id.keys(), yticklabels=category_to_id.keys())
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

#UPSBM    
    y_pred_sbm=clf.predict(tfidf_matrix_sbm)
    df=pd.DataFrame({"familyhead":x_sbm,"parent":xp_sbm,"religion":y_sbm,
                 "predicted":pd.Series(y_pred_sbm).map(id_to_category)})

    # df.to_csv(data_dir+'sbm_predictions_multiclass_'+classifier+'_concat_'+str(concat_model)+'.csv')

    print('SBM Accuracy of classifier level 1: {:.2f}'.format(accuracy_score(y_sbm=="Muslim",y_pred_sbm)))
    print(classification_report(y_sbm=="Muslim",y_pred_sbm,digits=4))
    conf_mat = confusion_matrix(y_sbm=="Muslim",y_pred_sbm)
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(conf_mat, annot=True, fmt='d',
                xticklabels={"nonMuslim","Muslim"}, yticklabels={"nonMuslim","Muslim"})
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

# Neural Model

### Utility Functions

In [ ]:
def tokenize(x,max_char):
  unique_symbols = Counter()

  for _, name in x.iteritems():
      unique_symbols.update(name)

  num_unique_symbols = len(unique_symbols) #- len(uncommon_symbols) + 1 
  print("Unique symbols:",num_unique_symbols, unique_symbols)

  tokenizer = Tokenizer(
      char_level=True,
      filters=None,
      oov_token='*',
      lower=False,
      num_words=None
  )

  tokenizer.fit_on_texts([x])
  char_dict = {}
  ind_to_char={}
  for i, char in enumerate(unique_symbols):
    char_dict[char] = i + 1
    ind_to_char[i+1]=char
  tokenizer.word_index = char_dict.copy()
  print(char_dict)
  
  tokenizer.word_index[tokenizer.oov_token] = max(char_dict.values()) + 1
  ind_to_char[max(char_dict.values()) + 1]=tokenizer.oov_token
  max_char=max(map(len,x))
  x_sequences = tokenizer.texts_to_sequences(x)
  with open(data_dir+"tokenizer.pkl", "wb") as f:
      pickle.dump((tokenizer,max_char), f)
  
  return (num_unique_symbols+1,tokenizer)


def getMaxchar():
  data=pd.read_csv(data_dir+"REDS_cleaned.csv")

  data=  data[data["name"].notna()]
  data.pname = data.pname.replace(np.nan, '', regex=True)
  data=data[data["religion"].notna()]
  data=data[~data.religion.isin(["Others"])]#,"Jain","Buddhist"])]
  data.drop_duplicates(subset =["name","pname","sex","religion"],keep = "first", inplace = True)
  data.name.replace(" ","}{", regex=True,inplace=True)
  data.pname.replace(" ","}{", regex=True,inplace=True)
  data.name="{"+data.name.astype(str)+"}"
  data.pname="{"+data.pname.astype(str)+"}"
  if(concat_model):
    data.name="#"+data.name.astype(str)+'#'+data.pname.astype(str)+"#"
    # data.pname="#"+data.pname.astype(str)+"#"
  max_char=max(map(len,data.name))
  vocab_size,tokenizer=tokenize(data.name,max_char)
  return max_char,tokenizer,vocab_size

max_char,tokenizer,vocab_size=getMaxchar()

In [ ]:
def plot_hist(history):
  plt.figure(figsize=(12, 5))
  plt.subplot(1, 3, 1)
  plt.plot(history['loss'], label='train_loss')
  plt.plot(history['val_loss'], label='test_loss')
  plt.title('Training and validation Loss')
  plt.legend()
  plt.subplot(1, 3, 2)
  plt.plot(history['accuracy'], label='train_Accuracy')
  plt.plot(history['val_accuracy'], label='test_Accuracy')
  plt.title('Training and validation Accuracy')
  plt.legend()
  plt.show()

### CNN Classifier

In [ ]:
def CNNmodel(shape, embedding_dim=15, activation='tanh', drop_embed=0.1, dropout=.25, dense_units=200,dense_activation='relu',
                       kernel_sizes=[[1,50], [2, 100], [3,150], [5,200], [6,250]], initializer='glorot_uniform',
                       optimizer='Nadam', batch_norm=True):
  x_can = Input(shape=(shape[0],), dtype='int32')
  
  embedding = Embedding(shape[1], embedding_dim, input_length = shape[0], trainable = True, embeddings_initializer = initializer)(x_can)
  embedding = BatchNormalization()(embedding)
  embedding = Dropout(drop_embed)(embedding)
  embedding = SpatialDropout1D(drop_embed)(embedding)
  convs = []
  
  for [kernel_size, num_filter] in kernel_sizes:
    l_conv = Conv1D(filters = num_filter, kernel_initializer = initializer, kernel_size = kernel_size,
                    padding = 'valid', name = "Convolution"+str(kernel_size), activation = activation, strides = 1)(embedding)
    l_pool= GlobalMaxPool1D()(l_conv)
    l_pool=AlphaDropout(.05)(l_pool)
    convs.append(l_pool)

  x = Concatenate(axis=1)(convs)
  x = Dense(dense_units, kernel_initializer=initializer, activation=dense_activation)(x)
  x = Dropout(dropout)(x)  
  x = Dense(6, kernel_initializer=initializer)(x)
  x = Activation('softmax')(x)
  model = Model(x_can,x)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])#
  # print(model.summary())
  return model

### Hyperparameter Tuning

#### keras tuner: First hyperparameter tuner option

In [ ]:
class MyHyperModel(HyperModel):

    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        
        min_lr = hp.Float(name='min_lr',min_value=1e-5, max_value=1e-3, sampling='log')
        factor = hp.Float(name='factor',min_value=.5, max_value=.8, sampling='linear')

        epoch = hp.Int(name='epoch',min=20,max=80, sampling='linear')
        batch_size = Integer(low=256, high=512, name='batch_size', prior='log-uniform',)
        activation = Categorical(categories=['relu', 'tanh','elu'],
                                     name='activation')
        initializer = Categorical(categories=['he_uniform','glorot_uniform','he_normal'],
                                     name='initializer')
        batch_norm = Categorical(categories=[True,False],
                                     name='batch_norm')
        patience = Integer(low=3, high=5, name='patience')
        dropout = hp.Float(name="dropout",min_value=0,max_value=.5, sampling='linear')
        emb_dropout = hp.Float(name="emb_dropout",min_value=0,max_value=.25,sampling='linear')
        dense_units= hp.Int(name='units',min_value=0, max_value=400, sampling='linear')
        dense_act = hp.Choice(name='dense_act',values=['relu','tanh','sigmoid'])
        filter=[]
        for i in range(1,7):
          filter.append(hp.Int( name="filter"+str(i),min_value=0, max_value=300,sampling="linear"))

        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor,patience=3, min_lr=min_lr)
        model=CNNmodel(shape,vocab_size, activation='elu', drop_embed=emb_dropout, dropout=dropout, dense_units=dense_units,dense_activation=dense_act,
              kernel_sizes=[[1,filter[0]],[2,filter[1]], [3, filter[2]], [4,filter[3]], [5,filter[4]], [6,filter[5]]],#,[7,filter7]],#[8,filter8]],#,[9,filter9]],
              initializer='glorot_uniform',optimizer='Nadam', batch_norm=True)        
        return model

hypermodel = MyHyperModel(num_classes=1)    

In [ ]:
non_neural=False
x_train,x_val,x_test,xp_train,xp_val,xp_test,Y_train,Y_val,Y_test,encoder = load_reds_split()
for concat_model in [True,False]:
  if concat_model==False:
    df1=pd.DataFrame({"x":x_train,"y":Y_train})
    df2=pd.DataFrame({"x":xp_train,"y":Y_train})
    df2 = df2[df2.x!=""]
    df=pd.concat([df1,df2],axis=0)

    dfv1=pd.DataFrame({"x":x_val,"y":Y_val})
    dfv2=pd.DataFrame({"x":xp_val,"y":Y_val})
    dfv2=dfv2[dfv2["x"].notna()]
    dfv2 = dfv2[dfv2.x!=""]
    dfv=pd.concat([dfv1,dfv2],axis=0)
    x_train=df.x
    x_val=dfv.x
    Y_train=df.y
    Y_val=dfv.y
  Y_test = encoder.transform(Y_test)
  Y_val = encoder.transform(Y_val)
  Y_train = encoder.transform(Y_train)
  y_train_labels = np.argmax(Y_train, axis =1)
  class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train_labels), y_train_labels)

  padded_sequences_x=tokenizer.texts_to_sequences(x_train)
  xtrain=pad_sequences(padded_sequences_x, maxlen=max_char,padding='post', truncating='post')

  padded_sequences_x=tokenizer.texts_to_sequences(x_val)
  xval=pad_sequences(padded_sequences_x, maxlen=max_char,padding='post', truncating='post')

  padded_sequences_x=tokenizer.texts_to_sequences(x_test)
  xtest=pad_sequences(padded_sequences_x, maxlen=max_char,padding='post', truncating='post')

  class_weights = {i : class_weights[i] for i in range(6)}
  shape=(xtrain.shape[1], vocab_size)
  tuner = Hyperband(
    hypermodel,
    objective='val_loss',
    max_epochs=25,
    seed=10,
    project_name='multi_religion')


  # Search for the best parameters of the neural network using the contructed Hypberband tuner
  tuner.search(xtrain, Y_train,
              epochs=10, 
              validation_data=(xval, Y_val) )

  # Get the best hyperparameters from the search
  params = tuner.get_best_hyperparameters()[0]

  # Build the model using the best hyperparameters
  model = tuner.hypermodel.build(params)
  model_name = model_dir+'multiclass_cnn_'+str(concat_model)+'.h5'
  mc = ModelCheckpoint(model_name, monitor='val_loss', mode='min', save_best_only=True,verbose=1)

  # Train the best fitting model
  model.fit(xtrain, Y_train,validation_data=(xval, Y_val),initial_epoch =0, epochs = 80,
                        shuffle=True, batch_size = 512,verbose=0,callbacks=[reduce_lr,mc],class_weight=class_weights,
                        workers=-1, use_multiprocessing=True)

  # Check the accuracy plots
  hyperband_accuracy_df = pd.DataFrame(model.history.history)

  hyperband_accuracy_df[['loss', 'accuracy']].plot()
  plt.title('Loss & Accuracy Per EPOCH')
  plt.xlabel('EPOCH')
  plt.ylabel('Accruacy')
  plt.show()


#### skopt: Second hyperparameter tuner option

In [ ]:
# for concat_model in [True,False]:
min_lr = Real(low=1e-5, high=1e-3, prior='log-uniform',
                        name='min_lr')
factor = Real(low=.5, high=.8, prior='uniform',
                        name='factor')

epoch = Integer(low=20, high=80, name='epoch',prior='uniform')
batch_size = Integer(low=256, high=512, name='batch_size', prior='log-uniform',)
activation = Categorical(categories=['relu', 'tanh','elu'],
                             name='activation')
initializer = Categorical(categories=['he_uniform','glorot_uniform','he_normal'],
                             name='initializer')
batch_norm = Categorical(categories=[True,False],
                             name='batch_norm')
patience = Integer(low=3, high=5, name='patience')
dropout = Real(low=0,high=.5,name="dropout", prior='uniform')
emb_dropout = Real(low=0,high=.25,name="emb_dropout",prior='uniform')
dense_units= Integer(low=0, high=400, name="dense_units", prior='uniform')
dense_act = Categorical(categories=['relu','tanh','sigmoid'],name='dense_act')

dimensions = [min_lr,
              factor,
              epoch,
              batch_size,
              activation,
              initializer,
              dropout,
              emb_dropout,
              # batch_norm
              dense_units,
              dense_act
            ]
for i in range(1,7):
  dimensions.append(Integer(low=0, high=300, name="filter"+str(i)))

# print(dimensions)
default_parameters = [0.0003918194347141745,  0.7339073000818309, 56, 0.22291637642679563, 0.024993728954500728, 184, 'sigmoid', 43, 195, 17, 217, 282, 0]
best_loss=100
@use_named_args(dimensions=dimensions)
def fitness(min_lr,  factor, epoch, dropout, emb_dropout,dense_units,dense_act,filter1,filter2,filter3,filter4,filter5,filter6):#,filter7):
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor,patience=3, min_lr=min_lr)
    model=CNNmodel(shape,vocab_size, activation='elu', drop_embed=emb_dropout, dropout=dropout, dense_units=dense_units,dense_activation=dense_act,
          kernel_sizes=[[1,filter1], [2, filter2], [3,filter3], [4,filter4], [5,filter5]],#,[7,filter7]],#[8,filter8]],#,[9,filter9]],
          initializer='glorot_uniform',optimizer='Nadam', batch_norm=True)        
    mc = ModelCheckpoint(model_name, monitor='val_loss', mode='min', save_best_only=True,verbose=1)

    #named blackbox becuase it represents the structure
    blackbox = model.fit(xtrain, Y_train,validation_data=(xval, Y_val),initial_epoch =0, epochs = epoch,
                      shuffle=True, batch_size = 512,verbose=0,callbacks=[reduce_lr],class_weight=class_weights,
                      workers=-1, use_multiprocessing=True)
    
    #return the validation accuracy for the last epoch.
    loss = blackbox.history['val_loss'][-1]

    # Print the classification accuracy.
    print("loss: {0:.2%}".format(loss))

    # Save the model if it improves on the best-found performance.

    # We use the global keyword so we update the variable outside of this function.
    global best_loss
    if loss < best_loss:
        model.save(model_name)   
        best_loss = loss
    del model
    
    # Clear the Keras session, otherwise it will keep adding new models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    ops.reset_default_graph()
    print(min_lr,  factor, epoch, dropout, emb_dropout,dense_units,dense_act,filter1,filter2,filter3,filter4,filter5,filter6)
   
    # the optimizer aims for the lowest score, so we return our negative accuracy
    return loss

In [ ]:
for concat_model in [True, False]:
  non_neural=False
  x_train,x_val,x_test,xp_train,xp_val,xp_test,Y_train,Y_val,Y_test,encoder = load_reds_split()

  if concat_model==False:
    df1=pd.DataFrame({"x":x_train,"y":Y_train})
    df2=pd.DataFrame({"x":xp_train,"y":Y_train})
    df2 = df2[df2.x!=""]
    df=pd.concat([df1,df2],axis=0)

    dfv1=pd.DataFrame({"x":x_val,"y":Y_val})
    dfv2=pd.DataFrame({"x":xp_val,"y":Y_val})
    dfv2=dfv2[dfv2["x"].notna()]
    dfv2 = dfv2[dfv2.x!=""]
    dfv=pd.concat([dfv1,dfv2],axis=0)
    x_train=df.x
    x_val=dfv.x
    Y_train=df.y
    Y_val=dfv.y
  Y_test = encoder.transform(Y_test)
  Y_val = encoder.transform(Y_val)
  Y_train = encoder.transform(Y_train)
  y_train_labels = np.argmax(Y_train, axis =1)
  class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train_labels), y_train_labels)

  padded_sequences_x=tokenizer.texts_to_sequences(x_train)
  xtrain=pad_sequences(padded_sequences_x, maxlen=max_char,padding='post', truncating='post')

  padded_sequences_x=tokenizer.texts_to_sequences(x_val)
  xval=pad_sequences(padded_sequences_x, maxlen=max_char,padding='post', truncating='post')

  padded_sequences_x=tokenizer.texts_to_sequences(x_test)
  xtest=pad_sequences(padded_sequences_x, maxlen=max_char,padding='post', truncating='post')

  class_weights = {i : class_weights[i] for i in range(6)}
  shape=(xtrain.shape[1], vocab_size)

  gp_result = gp_minimize(func = fitness,
                              dimensions=dimensions,
                              n_calls=40,
                              acq_func='EI',
                              n_jobs=-1,
                              verbose=1,
                              x0=default_parameters,random_state=42  )
  plot_convergence(gp_result)

  search_result.x
  space.point_to_dict(gp_result.x)
  gp_result.fun
  sorted(zip(gp_result.func_vals, gp_result.x_iters))             

### Train

In [ ]:
non_neural=False
concat_model=True
model_name = model_dir+ 'multiclass_CNN'+str(concat_model)+'.h5'

In [ ]:
x_train,x_val,x_test,xp_train,xp_val,xp_test,Y_train,Y_val,Y_test,encoder = load_reds_split()

if concat_model==False:
  df1=pd.DataFrame({"x":x_train,"y":Y_train})
  df2=pd.DataFrame({"x":xp_train,"y":Y_train})
  df2 = df2[df2.x!=""]
  df=pd.concat([df1,df2],axis=0)

  dfv1=pd.DataFrame({"x":x_val,"y":Y_val})
  dfv2=pd.DataFrame({"x":xp_val,"y":Y_val})
  dfv2.dropna(subset=["x"],inplace=True)
  dfv2 = dfv2[dfv2.x!=""]
  dfv=pd.concat([dfv1,dfv2],axis=0)
  x_train=df.x
  x_val=dfv.x
  Y_train=df.y
  Y_val=dfv.y

Y_test = encoder.transform(Y_test)
Y_val = encoder.transform(Y_val)
Y_train = encoder.transform(Y_train)
y_train_labels = np.argmax(Y_train, axis =1)
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train_labels), y_train_labels)

padded_sequences_x=tokenizer.texts_to_sequences(x_train)
xtrain=pad_sequences(padded_sequences_x, maxlen=max_char,padding='post', truncating='post')

padded_sequences_x=tokenizer.texts_to_sequences(x_val)
xval=pad_sequences(padded_sequences_x, maxlen=max_char,padding='post', truncating='post')

padded_sequences_x=tokenizer.texts_to_sequences(x_test)
xtest=pad_sequences(padded_sequences_x, maxlen=max_char,padding='post', truncating='post')

class_weights = {i : class_weights[i] for i in range(6)}

shape=(xtrain.shape[1], vocab_size)

In [ ]:
patience=2
min_lr=0.0002
drop_embed=.01
activation='tanh'
dense_units=400
initializer='he_uniform'
epochs=80
kernel_sizes=[[1,50], [2,300], [3,305], [4,200], [5,250], [6,200],[7,200]]
if concat_model:
  patience=3
  min_lr=0.00027
  activation='elu'
  drop_embed=0.02
  dense_units=200
  dense_activation='sigmoid',
  kernel_sizes=[[1,239],[2,248], [3,100 ], [4,150], [5,150], [6,250],[7,200]]
  initializer='glorot_uniform'
  epochs=60

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor= factor ,patience=patience, min_lr= min_lr)

mc = ModelCheckpoint(model_name, monitor='val_loss', mode='min', save_best_only=True,verbose=1)
model=CNNmodel(shape,vocab_size, activation=activation, drop_embed=drop_embed, dropout=.02, dense_units=dense_units,dense_activation='sigmoid',
              kernel_sizes=kernel_sizes,initializer=initializer,optimizer='Nadam', batch_norm=True)        
        
history=model.fit(xtrain, Y_train,validation_data=(xval, Y_val),initial_epoch =0, epochs =epochs,
                      shuffle=True, batch_size = 512,verbose=1,callbacks=[mc,reduce_lr],class_weight=class_weights,
                      workers=-1, use_multiprocessing=True)
plot_hist(history.history)

### Test

In [ ]:
model=load_model(model_name)
_,x_val,x_test,_,xp_val,xp_test,_,Y_val,Y_test,encoder = load_reds_split()
x_sbm,xp_sbm,y_sbm=load_upsbm()

###########VAL
padded_sequences_xval=tokenizer.texts_to_sequences(x_val)
xval=pad_sequences(padded_sequences_xval, maxlen=max_char,padding='post', truncating='post')
y_pred_val=model.predict(xval,batch_size=1024,verbose=1,use_multiprocessing=True,workers=-1)
df=pd.DataFrame({"familyhead":x_val,"parent":xp_val,"religion":Y_val,
              "predicted":pd.Series(encoder.inverse_transform(y_pred_val))})
# df.to_csv(data_dir+'val2_predictions_multiclass_CNN_concat_'+str(concat_model)+'.csv')

print('REDS in SAMPLE Accuracy of classifier level 1: {:.2f}'.format(accuracy_score(Y_val,encoder.inverse_transform(y_pred_val))))
print(classification_report(Y_val, encoder.inverse_transform(y_pred_val),digits=4,target_names=encoder.classes_))
conf_mat = confusion_matrix(Y_val,encoder.inverse_transform(y_pred_val))
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

############ REDSTEST
padded_sequences_xtest=tokenizer.texts_to_sequences(x_test)
xtest=pad_sequences(padded_sequences_xtest, maxlen=max_char,padding='post', truncating='post')
y_pred_test=model.predict(xtest,batch_size=1024,verbose=1,use_multiprocessing=True,workers=-1)
df=pd.DataFrame({"familyhead":x_test,"parent":xp_test,"religion":pd.Series(Y_test),
                 "pred":pd.Series(encoder.inverse_transform(y_pred_test))})
# df.to_csv(data_dir+'test2_predictions_multiclass_CNN_concat_'+str(concat_model)+'.csv')

print('REDS out of SAMPLE Accuracy of classifier level 1: {:.2f}'.format(accuracy_score(Y_test,encoder.inverse_transform(y_pred_test))))
print(classification_report(Y_test, encoder.inverse_transform(y_pred_test),digits=4,target_names=encoder.classes_))
conf_mat = confusion_matrix(Y_test,encoder.inverse_transform(y_pred_test))
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

############ UPSBM TEST
padded_sequences_xsbm=tokenizer.texts_to_sequences(x_sbm)
xsbm=pad_sequences(padded_sequences_xsbm, maxlen=max_char,padding='post', truncating='post')
y_pred_sbm=model.predict(xsbm,batch_size=1024,verbose=1,use_multiprocessing=True,workers=-1)
df=pd.DataFrame({"familyhead":x_sbm,"parent":xp_sbm,"religion":y_sbm,
              "predicted":pd.Series(encoder.inverse_transform(y_pred_sbm))})
# df.to_csv(data_dir+'sbm2_predictions_multiclass_CNN_concat_'+str(concat_model)+'.csv')

print('UPSBM out of SAMPLE Accuracy of classifier level 1: {:.2f}'.format(accuracy_score(y_sbm=="Muslim",encoder.inverse_transform(y_pred_sbm)=="Muslim")))
print(classification_report(y_sbm=="Muslim", encoder.inverse_transform(y_pred_sbm)=="Muslim",digits=4,target_names=['non-muslim','muslim']))
conf_mat = confusion_matrix(y_sbm=="Muslim",encoder.inverse_transform(y_pred_sbm)=="Muslim")
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d', xticklabels=['non-muslim','muslim'], yticklabels=['non-muslim','muslim'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()